# {mod}`ultralytics` 训练模式

训练模式：在自定义或预载数据集上对模型进行微调。

In [ ]:
import set_env

训练深度学习模型包括向其输入数据并调整参数，以便能够进行准确的预测。Ultralytics YOLOv11 中的训练模式专为高效和有效地训练目标检测模型而设计，充分利用了现代硬件的能力。本指南旨在涵盖您开始使用 YOLOv11 的全面功能集训练自己的模型所需的所有详细信息。

```{admonition} 为什么选择 Ultralytics YOLO 进行模型训练？
- 效率：充分利用硬件，无论您是使用单个 GPU 设置还是在多个 GPU 之间扩展。
- 多功能性：除 COCO、VOC 和 ImageNet 等现成数据集外，还可在自定义数据集上进行训练。
- 用户友好型：简单而强大的 CLI 和 Python 界面，提供直接的培训体验。
- 超参数灵活性：可定制的超参数范围广泛，可对模型性能进行微调。
```

```{admonition} 训练模式的主要功能
- 自动下载数据集：首次使用时会自动下载 COCO、VOC 和 ImageNet 等标准数据集。
支持多个 GPU ：在多个 GPU 上无缝扩展您的培训工作，以加快进程。
- 超参数配置：通过 YAML 配置文件或CLI 参数修改超参数的选项。
- 可视化和监控：实时跟踪培训指标和可视化学习过程，以获得更好的洞察力。
```

在 COCO8 数据集上，使用 640 的图像大小对 YOLOv11n 进行 100 个 epoch 的训练。可以通过 `device` 参数指定训练设备。如果没有传递参数且 GPU可用，则 `device=0` 将被使用，否则 `device='cpu'` 将被使用。

```python
from ultralytics import YOLO
# 加载模型
model = YOLO("yolo11n.yaml")  # 利用 YAML 构建新模型
model = YOLO("yolo11n.pt")  # 加载预训练模型（推荐用于训练）
model = YOLO("yolo11n.yaml").load("yolo11n.pt") # 从 YAML 构建并迁移权重
# 训练模型
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)
```

## 多 GPU 训练

通过将训练负载分配到多个 GPU 上，多GPU训练能够更有效地利用可用的硬件资源。这一特性既可以通过 Python API 也可以通过命令行界面来实现。为了启用多 GPU 训练，您需要指定希望使用的 GPU 设备 ID。

要使用 2 个 GPU（CUDA 设备 0 和 1）进行训练，请使用以下命令。根据需要扩展到其他 GPU。

```python
from ultralytics import YOLO
# 加载模型
model = YOLO("yolo11n.pt")  # 加载预训练模型（建议用于训练）
# 使用 2 个 GPU 训练
results = model.train(data="coco8.yaml", epochs=100, imgsz=640, device=[0, 1])
```

## 苹果硅 MPS 训练

随着对 Ultralytics YOLO 模型中集成的苹果硅芯片的支持，现在可以在利用强大的 Metal Performance Shaders (MPS) 框架的设备上训练您的模型。MPS 为在苹果定制硅上执行计算和图像处理任务提供了高性能的方式。

为了在苹果硅芯片上启用训练，您应该在启动训练过程时指定 `'mps'` 作为您的设备。以下是如何在 Python 中以及通过命令行进行操作的示例：
```python
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model with MPS
results = model.train(data="coco8.yaml", epochs=100, imgsz=640, device="mps")
```

利用苹果硅芯片的计算能力，可以更高效地处理训练任务。如需更详细的指导和高级配置选项，请参阅 [PyTorch MPS 文档](https://pytorch.org/docs/stable/notes/mps.html)。

## 恢复中断的训练

在处理深度学习模型时，从之前保存的状态恢复训练其关键特性。这在多种情况下都非常有用，例如当训练过程意外中断，或者您希望使用新数据或更多周期继续训练模型时。

当训练恢复时，Ultralytics YOLO 会加载最后一次保存的模型权重，并恢复优化器状态、学习率调度器和周期数。这样，您就可以无缝地从停止的地方继续训练过程。

在 Ultralytics YOLO 中，您可以通过在调用 `train` 方法时将 `resume` 参数设置为 `True`，并指定包含部分训练模型权重的 `.pt` 文件路径来轻松恢复训练。

下面是使用 Python 以及通过命令行恢复中断训练的示例：

```python 
from ultralytics import YOLO
# Load a model
model = YOLO("path/to/last.pt")  # load a partially trained model
# Resume training
results = model.train(resume=True)
```

通过设置 `resume=True`，`train` 函数将从中断的地方继续训练，利用存储在 `'path/to/last.pt'` 文件中的状态。如果省略 `resume` 参数或将其设置为 `False`，则训练函数将从零开始训练。

请牢记，默认情况下检查点是在每个时期的末尾保存的，或者使用 `save_period` 参数在固定间隔进行保存。因此，您至少需要完成一个 epoch 才能恢复训练运行。

## 训练设置

YOLO 模型的训练设置包括在训练过程中使用的各种超参数和配置。这些设置影响模型的性能、速度和准确性（参见链接）。关键的训练设置包括批量大小、学习率、动量和权重衰减。此外，优化器的选择、损失函数以及训练数据集的构成也会影响训练过程。对这些设置进行仔细调整和实验对于优化性能至关重要。

好的，我会尽力提供准确并学术性的翻译。以下是您提供的段落的中文翻译：

| 参数         | 默认值 | 描述                                                                                                                                                                                                                                                                 |
| ------------ | ------ | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `model`      | `None` | 指定用于训练的模型文件。可以是预训练模型的路径（`.pt`）或配置文件（`.yaml`）。这对于定义模型结构或初始化权重至关重要。                                                                                                             |
| `data`       | `None` | 数据集配置文件的路径（如`coco8.yaml`）。该文件包含特定于数据集的参数，包括训练和验证数据的路径、类别名称以及类别数量。                                                                                                         |
| `epochs`     | `100`  | 总训练周期数。每个周期表示对整个数据集的一次完整遍历。调整此值可以影响训练持续时间和模型性能。                                                                                                                  |
| `time`       | `None` | 最大训练时间（以小时为单位）。如果设置了此值，则覆盖`epochs`参数，允许在指定时间后自动停止训练。这在时间受限的训练场景中很有用。                                                                                                  |
| `patience`   | `100`  | 在验证指标没有改善的情况下等待的周期数，之后提前停止训练。通过在性能停滞时停止训练来防止过拟合。                                                                                                                    |
| `batch`      | `16`   | 批量大小，有三种模式：设置为整数（例如，`batch=16`），自动模式为60%的GPU内存利用率（`batch=-1`），或带有指定利用率分数的自动模式（`batch=0.70`）。                                                                                             |
| `imgsz`      | `640`  | 训练的目标图像尺寸。所有图像在此维度上调整大小后再输入到模型中。影响模型的准确性和计算复杂性。                                                                                                                  |
| `save`       | `True` | 启用保存训练检查点和最终模型权重。对于恢复训练或模型部署非常有用。                                                                                                                                      |
| `save_period`| `-1`   | 保存模型检查点的频率，以周期数指定。值为-1将禁用此功能。在长时间的训练会话中保存中间模型非常有用。                                                                                                                      |
| `cache`      | `False`| 启用数据集图像缓存，可以选择在内存中（`True`/`ram`）、磁盘上（`disk`）或禁用它（`False`）。通过减少磁盘I/O提高训练速度，但会增加内存使用量。                                                                                              |
| `device`     | `None` | 指定用于训练的计算设备：单个GPU（`device=0`）、多个GPU（`device=0,1`）、CPU（`device=cpu`）或苹果硅片的MPS（`device=mps`）。                                                                                                |
| `workers`    | `8`    | 数据加载的工作线程数（如果是多GPU训练，则为每个`RANK`）。影响数据预处理和模型输入的速度，在多GPU设置中特别有用。                                                                                                       |
| `project`    | `None` | 存储训练输出的项目目录名称。允许有组织地存储不同的实验。                                                                                                                                               |
| `name`       | `None` | 训练运行的名称。用于在项目文件夹中创建一个子目录，其中存储训练日志和输出。                                                                                                                                  |
| `exist_ok`   | `False`| 如果为True，允许覆盖现有的项目/名称目录。在不需要手动清除以前的输出的情况下进行迭代实验非常有用。                                                                                                                     |
| `pretrained` | `True` | 确定是否从预训练模型开始训练。可以是布尔值或特定模型的字符串路径，从中加载权重。提高了训练效率和模型性能。                                                                                                       |
| `optimizer`  | `'auto'`| 训练选择的优化器。选项包括`SGD`、`Adam`、`AdamW`、`NAdam`、`RAdam`、`RMSProp`等，或者根据模型配置自动选择（`auto`）。影响收敛速度和稳定性。                                                                            |
| `verbose`    | `False`| 启用详细输出，提供详细的日志和进度更新。对于调试和密切监控训练过程非常有用。                                                                                                                                   |
| `seed`       | `0`    | 设置训练的随机种子，确保在相同配置下结果的可重复性。                                                                                                                                               |
| `deterministic`| `True` | 强制使用确定性算法，以确保可重复性，但由于限制了非确定性算法，可能会影响性能和速度。                                                                                                                                |
| `single_cls` | `False`| 在训练期间将所有多类数据集中的类别视为单一类别。对于二分类任务或关注对象存在而不是分类时非常有用。                                                                                                      |
| `rect`       | `False`| 启用矩形训练，优化批次组合以最小化填充。可以提高效率和速度，但可能会影响模型准确性。                                                                                                                    |
| `cos_lr`     | `False` | 使用余弦学习率调度器，根据余弦曲线调整每个周期的学习率。这有助于更好地收敛学习率的管理。                                                                                                                                                                                                                       |
| `close_mosaic` | `10`   | 在最后N个周期内禁用数据增强的马赛克方法以稳定训练结束前的训练。设置为0则禁用此功能。                                                                                                                                                                                                                           |
| `resume`     | `False` | 从上次保存的检查点恢复训练。自动加载模型权重、优化器状态和周期计数，无缝继续训练。                                                                                                                                                                                                                        |
| `amp`        | `True`  | 启用自动混合精度（AMP）训练，减少内存使用并可能加快训练速度，对准确性的影响最小。                                                                                                                                                                                                                          |
| `fraction`   | `1.0`   | 指定用于训练的数据集部分。允许在完整数据集的子集上进行训练，这对于实验或资源有限时非常有用。                                                                                                                                                                                                                    |
| `profile`    | `False` | 启用ONNX和TensorRT速度的剖析，有助于优化模型部署。                                                                                                                                                                                                                                                           |
| `freeze`     | `None`  | 冻结模型的前N层或通过索引指定的层，减少可训练参数的数量。对于微调或迁移学习非常有用。                                                                                                                                                                                                                                |
| `lr0`        | `0.01`  | 初始学习率（即`SGD=1E-2`, `Adam=1E-3`）。调整这个值对优化过程至关重要，影响模型权重更新的速度。                                                                                                                                                                                                              |
| `lrf`        | `0.01`  | 最终学习率为初始学习率的一部分 = (`lr0 * lrf`)，与调度器一起使用以随时间调整学习率。                                                                                                                                                                                                                                |
| `momentum`   | `0.937` | SGD的动量因子或Adam优化器的beta1，影响当前更新中过去梯度的结合方式。                                                                                                                                                                                                                                     |
| `weight_decay` | `0.0005` | L2正则化项，惩罚大权重以防止过拟合。                                                                                                                                                                                                                                                                     |
| `warmup_epochs` | `3.0`  | 学习率预热的周期数，将学习率从低值逐渐增加到初始学习率以稳定早期训练。                                                                                                                                                                                                                                  |
| `warmup_momentum` | `0.8`  | 预热阶段的初始动量，逐渐调整到设置的动量值。                                                                                                                                                                                                                                                                   |
| `warmup_bias_lr` | `0.1`  | 预热阶段偏置参数的学习率，帮助在初期周期稳定模型训练。                                                                                                                                                                                                                                                             |
| `box`        | `7.5`   | 损失函数中框损失分量的权重，影响对预测边界框坐标的重视程度。                                                                                                                                                                                                                                                           |
| `cls`        | `0.5`   | 总损失函数中分类损失的权重，影响正确类别预测的重要性相对于其他组件。                                                                                                                                                                                                                                                       |
| `dfl`        | `1.5`   | 某些YOLO版本中使用的分布焦点损失的权重，用于细粒度分类。                                                                                                                                                                                                                                                                 |
| `pose`       | `12.0`  | 姿态估计模型中姿态损失的权重，影响对准确预测姿态关键点的重视程度。                                                                                                                                                                                                                                                           |
| `kobj`       | `2.0`   | 姿态估计模型中关键点对象性损失的权重，平衡检测置信度和姿态精度。                                                                                                                                                                                                                                                             |
| `label_smoothing` | `0.0`  | 应用标签平滑，将硬标签软化为目标标签和均匀分布的混合，可以改善泛化能力。                                                                                                                                                                                                                                   |
| `nbs`        | `64`    | 归一化损失的名义批量大小。                                                                                                                                                                                                                                                                           |
| `overlap_mask` | `True` | 决定是否将对象掩码合并为单个掩码用于训练，或者在每个对象上保持单独的掩码。如果存在重叠，较小的掩码将在合并过程中覆盖在较大的掩码上。
| `mask_ratio` | `4` | 分割掩码的下采样比率，影响训练期间使用的掩码分辨率。
| `dropout` | `0.0` | 分类任务中的正则化丢弃率，通过在训练期间随机省略单元来防止过拟合。
| `val` | `True` | 在训练期间启用验证，允许定期评估模型在独立数据集上的性能。
| `plots` | `False` | 生成并保存训练和验证指标的图表以及预测示例，提供对模型性能和学习进度的视觉洞察。                                                                                     

```{admonition} 批处理大小设置说明

批处理参数可通过以下三种方式配置：

- 固定[批处理大小](https://www.ultralytics.com/glossary/batch-size)：设置一个整数值（例如，`batch=16`），直接指定每批的图像数量。
- 自动模式（$60\%$ GPU 内存）：使用 `batch=-1` 以自动调整批处理大小，大约利用 $60\%$ 的 CUDA 内存。
- 带利用率分数的自动模式：设置分数值（例如，`batch=0.70`），根据指定的 GPU 内存使用比例来调整批处理大小。
```

## 增强设置和超参数

增强技术对于提升 YOLO 模型的鲁棒性和性能至关重要，它通过向训练数据引入变异性，帮助模型更好地泛化到未见数据。下表概述了每个增强参数的目的和效果：

**参数说明：**

| 参数名         | 类型    | 默认值     | 范围           | 描述                                                                                                                                                                                       |
|---------------|---------|------------|----------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `hsv_h`       | `float` | `0.015`    | `0.0 - 1.0`     | 通过调整图像色调的一个分数，引入颜色变化，帮助模型在不同光照条件下泛化。                                                                                               |
| `hsv_s`       | `float` | `0.7`      | `0.0 - 1.0`     | 通过一个分数改变图像的饱和度，影响颜色强度。适用于模拟不同环境条件。                                                                                                 |
| `hsv_v`       | `float` | `0.4`      | `0.0 - 1.0`     | 通过一个分数修改图像的亮度（明暗），帮助模型在各种光照条件下表现良好。                                                                                               |
| `degrees`     | `float` | `0.0`      | `-180 - +180`   | 随机旋转图像到指定度数范围内，提高模型识别不同方向物体的能力。                                                                                                     |
| `translate`   | `float` | `0.1`      | `0.0 - 1.0`     | 按图像尺寸的分数水平和垂直移动图像，有助于学习检测部分可见的物体。                                                                                                   |
| `scale`       | `float` | `0.5`      | `>=0.0`        | 按增益因子缩放图像，模拟物体与相机的距离差异。                                                                                                                   |
| `shear`       | `float` | `0.0`      | `-180 - +180`   | 按指定角度剪切图像，模仿从不同角度观察物体的效果。                                                                                                              |
| `perspective` | `float` | `0.0`      | `0.0 - 0.001`   | 对图像应用随机透视变换，增强模型理解三维空间中物体的能力。                                                                                                        |
| `flipud`      | `float` | `0.0`      | `0.0 - 1.0`     | 以指定概率将图像上下翻转，增加数据多样性而不改变物体特性。                                                                                                         |
| `fliplr`      | `float` | `0.5`      | `0.0 - 1.0`     | 以指定概率将图像左右翻转，有助于学习对称物体和增加数据集多样性。                                                                                                   |
| `bgr`         | `float` | `0.0`      | `0.0 - 1.0`     | 以指定概率将图像通道从RGB转换为BGR，有助于提高对错误通道顺序的鲁棒性。                                                                                             |
| `mosaic`      | `float` | `1.0`      | `0.0 - 1.0`     | 将四张训练图像合并为一张，模拟不同场景组合和物体交互。对于复杂场景理解非常有效。                                                                                   |
| `mixup`       | `float` | `0.0`      | `0.0 - 1.0`     | 混合两张图像及其标签，创建一个合成图像。通过引入标签噪声和视觉变化，增强模型的泛化能力。                                                                             |
| `copy_paste`  | `float` | `0.0`      | `0.0 - 1.0`     | 将一个图像中的物体复制并粘贴到另一个图像中，有助于增加物体实例和学习物体遮挡。                                                                                        |
| `copy_paste_mode` | `str`   | `flip`     | -              | 选择复制粘贴增强方法，选项包括 (`"flip"`, `"mixup"`)。                                                                                                              |
| `auto_augment`| `str`   | `randaugment`| -              | 自动应用预定义增强策略 (`randaugment`, `autoaugment`, `augmix`)，通过多样化视觉特征优化分类任务。                                                                    |
| `erasing`     | `float` | `0.4`      | `0.0 - 0.9`     | 在分类训练过程中随机擦除图像的一部分，鼓励模型关注不太明显的特征进行识别。                                                                                             |
| `crop_fraction`| `float` | `1.0`      | `0.1 - 1.0`     | 裁剪分类图像到其大小的一个分数，强调中心特征并适应物体尺度，减少背景干扰。                                                                                             |

这些设置可以根据数据集和当前任务的具体要求进行调整。尝试不同的数值有助于找到最优的数据增强策略，从而提升模型的性能表现。

## 训练日志

在训练 YOLOv11 模型时，跟踪模型随时间的性能变化是非常有价值的。这时就需要用到日志记录功能。Ultralytics 的 YOLO 支持三种类型的日志记录器 - Comet、ClearML 和 TensorBoard。

### Comet

[Comet](https://docs.ultralytics.com/integrations/comet/#comet-ml)平台，它允许数据科学家和开发人员追踪、比较、解释和优化实验及模型。它提供了实时指标、代码差异对比以及超参数跟踪等功能。

```python
# pip install comet_ml
import comet_ml

comet_ml.init()
```

请记得在 Comet 网站的个人账户中登录，并获取您的 API 密钥。您需要将此密钥添加到环境变量或脚本中，以便记录您的实验数据。

### ClearML

[ClearML](https://clear.ml/) 开源平台，它自动化跟踪实验并帮助高效共享资源。该平台旨在帮助团队更有效地管理、执行和复现他们的机器学习工作。

```python
# pip install clearml
import clearml

clearml.browser_login()
```

执行此脚本后，您需要在浏览器上登录您的 ClearML 账户并验证会话。

### TensorBoard

[TensorBoard](https://www.tensorflow.org/tensorboard)是用于[TensorFlow](https://www.ultralytics.com/glossary/tensorflow)的可视化工具包。它允许你可视化你的TensorFlow图，绘制关于图执行的定量指标图表，并显示通过它的额外数据，如图像。

要在[Google Colab](https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/tutorial.ipynb)中使用TensorBoard：
```bash
load_ext tensorboard
tensorboard --logdir ultralytics/runs  # replace with 'runs' directory
```

要本地运行 TensorBoard，请执行以下命令并在 <http://localhost:6006/> 查看结果。

```bash
tensorboard --logdir ultralytics/runs  # replace with 'runs' directory
```

这将启动 TensorBoard，并将其指向您保存训练日志的目录。

配置好记录器后，您可以继续进行模型训练。所有训练指标都将自动记录在您选择的平台中，您可以访问这些日志来监控模型随时间的性能变化，比较不同模型，并识别改进的空间。